In [ ]:
import numpy as np
import ipyvolume as ipv
import symfit as sf

import ectopylasm as ep

In [ ]:
xyz = np.array((np.random.random(1000), np.random.normal(0, 0.01, 1000), np.random.random(1000)))

In [ ]:
ipv.clear()
ipv.scatter(*xyz, marker='circle_2d')
ipv.show()

In [ ]:
a, b, c, x0, y0, z0 = sf.parameters('a, b, c, x0, y0, z0')
x, y, z = sf.variables('x, y, z')
plane_model = {x: (x0 * a + y0 * b + z0 * c - y * b - z * c) / a}

In [ ]:
plane_fit = sf.Fit(plane_model, x=xyz[0], y=xyz[1], z=xyz[2])

In [ ]:
plane_fit_result = plane_fit.execute()

In [ ]:
print(plane_fit_result)

In [ ]:
p_fit = plane_fit_result.params

In [ ]:
ep.plot_plane((p_fit['x0'], p_fit['y0'], p_fit['z0']), (p_fit['a'], p_fit['b'], p_fit['c']), (0, 1), (0, 1))

That's not really a great fit. y0 should be about 0, certainly not 0.75. Also the stds seem weird and chi_squared is high.

Let's try again with initial values for x0, y0 and z0. We can set it to any of our random points.

In [ ]:
initial_guess = xyz.T[0]

a, b, c, x0, y0, z0 = sf.parameters('a, b, c, x0, y0, z0')
x0.value = initial_guess[0]
y0.value = initial_guess[1]
z0.value = initial_guess[2]
x, y, z = sf.variables('x, y, z')
plane_model = {x: (x0 * a + y0 * b + z0 * c - y * b - z * c) / a}

plane_fit = sf.Fit(plane_model, x=xyz[0], y=xyz[1], z=xyz[2])

plane_fit_result = plane_fit.execute()

print(plane_fit_result)

Hmm, weird.

In [ ]:
p_fit = plane_fit_result.params

In [ ]:
ep.plot_plane((p_fit['x0'], p_fit['y0'], p_fit['z0']), (p_fit['a'], p_fit['b'], p_fit['c']), (0, 1), (0, 1))

Hmm, ok, it's actually not totally off, at least it goes through the actual plane of points. The angle is just pretty much off.

Let's try including some limits, because x and z are also waaaaay way out there.

In [ ]:
initial_guess = xyz.T[0]

a, b, c, x0, y0, z0 = sf.parameters('a, b, c, x0, y0, z0')
x0.value = initial_guess[0]
x0.min, x0.max = (0, 1)
y0.value = initial_guess[1]
z0.value = initial_guess[2]
z0.min, z0.max = (0, 1)
x, y, z = sf.variables('x, y, z')
plane_model = {x: (x0 * a + y0 * b + z0 * c - y * b - z * c) / a}

plane_fit = sf.Fit(plane_model, x=xyz[0], y=xyz[1], z=xyz[2])

plane_fit_result = plane_fit.execute()

print(plane_fit_result)

Again, pretty crappy.

In [ ]:
p_fit = plane_fit_result.params
ep.plot_plane((p_fit['x0'], p_fit['y0'], p_fit['z0']), (p_fit['a'], p_fit['b'], p_fit['c']), (0, 1), (0, 1))

Let's try with initial values for a b c as well that together I think should be a pretty good fit already.

In [ ]:
initial_guess = xyz.T[0]

a, b, c, x0, y0, z0 = sf.parameters('a, b, c, x0, y0, z0')
a.value = 0.0001
b.value = 1
c.value = 0.0001
x0.value = initial_guess[0]
x0.min, x0.max = (0, 1)
y0.value = initial_guess[1]
z0.value = initial_guess[2]
z0.min, z0.max = (0, 1)
x, y, z = sf.variables('x, y, z')
plane_model = {x: (x0 * a + y0 * b + z0 * c - y * b - z * c) / a}

plane_fit = sf.Fit(plane_model, x=xyz[0], y=xyz[1], z=xyz[2])

plane_fit_result = plane_fit.execute()

print(plane_fit_result)

In [ ]:
p_fit = plane_fit_result.params
ep.plot_plane((p_fit['x0'], p_fit['y0'], p_fit['z0']), (p_fit['a'], p_fit['b'], p_fit['c']), (0, 1), (0, 1))

Crap!

Perhaps I should try to parameterize the plane differently... Is the division by a a problem, because it will give division by zero?

In [ ]:
initial_guess = xyz.T[0]

a, b, c, x0, y0, z0 = sf.parameters('a, b, c, x0, y0, z0')
a.value = 0
b.value = 1
c.value = 0
x0.value = initial_guess[0]
x0.min, x0.max = (0, 1)
y0.value = initial_guess[1]
z0.value = initial_guess[2]
z0.min, z0.max = (0, 1)
x, y, z = sf.variables('x, y, z')
plane_model = {y: (x0 * a + y0 * b + z0 * c - x * a - z * c) / b}

plane_fit = sf.Fit(plane_model, x=xyz[0], y=xyz[1], z=xyz[2])

plane_fit_result = plane_fit.execute()

print(plane_fit_result)

Ahhh, that was it! Coolio.

In [ ]:
ipv.clear()
ipv.scatter(*xyz, marker='circle_2d')
p_fit = plane_fit_result.params
ep.plot_plane((p_fit['x0'], p_fit['y0'], p_fit['z0']), (p_fit['a'], p_fit['b'], p_fit['c']), (0, 1), (0, 1))
ipv.show()